In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict

class SearchAndGenerate:
    def __init__(self):
        # BGE-M3 모델 로드
        self.embedding = SentenceTransformer('../bge-m3')
        
        # Qwen2.5-1.5B 모델 로드
        self.tokenizer = AutoTokenizer.from_pretrained("../Qwen2.5-1.5B-Instruct/")
        self.model = AutoModelForCausalLM.from_pretrained(
            "../Qwen2.5-1.5B-Instruct/",
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
    def encode_documents(self, documents: List[str]) -> np.ndarray:
        """문서들을 임베딩으로 변환"""
        return self.embedding.encode(documents, normalize_embeddings=True)
    
    def search(self, query: str, documents: List[str], top_k: int = 2) -> List[Dict]:
        """쿼리에 가장 관련성이 높은 문서들을 검색"""
        # 쿼리 임베딩 생성
        query_embedding = self.embedding.encode(query, normalize_embeddings=True)
        
        # 문서 임베딩 생성
        doc_embeddings = self.encode_documents(documents)
        
        # 유사도 계산
        similarities = np.dot(doc_embeddings, query_embedding)
        
        # 상위 k개 문서 선택
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        results = []
        for idx in top_indices:
            results.append({
                "document": documents[idx],
                "score": float(similarities[idx])
            })
            
        return results
    
    def generate_answer(self, query: str, context: str) -> str:
        """주어진 컨텍스트를 바탕으로 답변 생성"""
        prompt = f"""다음 질문에 대해 주어진 컨텍스트를 바탕으로 답변해주세요.

질문: {query}

컨텍스트: {context}

답변:"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )
            
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 프롬프트 부분 제거
        # answer = answer[len(prompt):].strip()
        
        return answer
    
    def search_and_generate(self, query: str, documents: List[str], top_k: int = 3) -> str:
        """검색과 답변 생성을 한 번에 수행"""
        # 관련 문서 검색
        search_results = self.search(query, documents, top_k)
        
        # 검색된 문서들을 컨텍스트로 결합
        context = "\n".join([result["document"] for result in search_results])
        
        # 답변 생성
        answer = self.generate_answer(query, context)
        
        return answer

In [6]:
system = SearchAndGenerate()

# 샘플 문서
documents = [
    "인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 컴퓨터 프로그램으로 구현한 기술입니다.",
    "머신러닝은 인공지능의 한 분야로, 데이터로부터 학습하여 예측이나 결정을 수행하는 알고리즘을 개발하는 기술입니다.",
    "딥러닝은 머신러닝의 한 종류로, 인공신경망을 사용하여 복잡한 패턴을 학습하는 기술입니다."
]

# 샘플 쿼리
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"

# 검색 및 답변 생성
answer = system.search_and_generate(query, documents)
print(f"질문: {query}")
print(f"답변: {answer}") 

질문: 인공지능과 머신러닝의 차이점은 무엇인가요?
답변: 다음 질문에 대해 주어진 컨텍스트를 바탕으로 답변해주세요.

질문: 인공지능과 머신러닝의 차이점은 무엇인가요?

컨텍스트: 머신러닝은 인공지능의 한 분야로, 데이터로부터 학습하여 예측이나 결정을 수행하는 알고리즘을 개발하는 기술입니다.
인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 컴퓨터 프로그램으로 구현한 기술입니다.
딥러닝은 머신러닝의 한 종류로, 인공신경망을 사용하여 복잡한 패턴을 학습하는 기술입니다.

답변: 인공지능(AI)과 머신러닝(ML) 사이에는 많은 차이점이 있습니다. 

1. 개념적인 차이: 인공지능은 모든 형태의 문제 해결 능력을 가진 기계적 동작을 의미하며, 이는 모든 것을 포함할 수 있습니다. 반면에 머신러닝은 특정한 문제에 대한 해법을 찾기 위한 시스템이며, 이는 대부분의 경우 AI의 일부분으로 작용합니다.

2. 목적: 인공지능은 인간의 감정, 생각, 판단 등을 포함하여 모든 정보를 처리하고 이해하는 능력을 가지고 있어야 합니다. 반면에 머신러닝은 특정한 작업을 수행하기 위해 설계된 프로그래밍 기술입니다.

3. 학습 방법: 인공지능은 전체 데이터셋에서 학습하며, 그 결과로 모든 정보에 대해 깊이 있는 이해와 통찰력을 얻습니다. 반면에 머신러닝은 특정 문제에 대한 답을 찾기 위해 특정한 데이터 세트를 훈련시키며, 이를 통해 특성화된 모델을 생성합니다.

4. 활용 범위: 인공지능은 다양한 분야에서 사용되지만, 특히 의학, 교육, 금융 등에서는 빠르게 성장하고 있습니다. 반면에 머신러닝은 특정 분야에서의 효율성을 향상시키는데 큰 역할을 합니다. 예를 들어, 치매 진단이나 공급망 관리에서 머신러닝이 매우 효과적으로 적용될 수 있습니다.

5. 딥러닝: 인공지능의 한 부분으로, 머신러닝은 일반적으로 인공지능에 속합니다. 그러나 딥러닝은 머신러닝의 특별한 유형입니다. 딥러닝은 인공신경망을 사용하여 복잡한 패턴을 학습하는 방법을 설명합니다. 

따라서, 인공지